# UResNet for eye detection in M0 pictures

In [1]:
import os
import random
from PIL import Image
import torch
from torch import optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torch.utils.data import random_split
from tqdm import tqdm
from matplotlib import pyplot as plt
from torch.amp import autocast, GradScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision.transforms import functional as F
import torch.nn.functional as Fu
from skimage.measure import label, regionprops
from skimage.morphology import convex_hull_image
import numpy as np
import cv2
from scipy import ndimage

print(torch.__version__)          # should show 2.8.0+cu126
print(torch.version.cuda)         # should show '12.6'
print(torch.cuda.is_available())  # should be True
print(torch.cuda.get_device_name(0))  # NVIDIA RTX A6000

2.8.0+cu126
12.6
True
NVIDIA RTX A6000


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
num_workers = 1

if device == "cuda":
    print("Using GPU")
    num_workers = torch.cuda.device_count() * 4
else:
    print("Using CPU")

generator = torch.Generator(device=device)

Using GPU


In [3]:
DATA_DIR = "data/"
DATASET_DIR = DATA_DIR + "optic_disk/"
TRAIN_DIR = DATASET_DIR + "train/"
TRAIN_MASKS_DIR = DATASET_DIR + "train_masks/"
LIMIT = None

STATES_DIR = "states/"
MODEL_PATH = STATES_DIR + "optic_disk_11.pth"
IMAGE_SIZE = 1024

In [4]:
LEARNING_RATE = 1e-3
BATCH_SIZE = 8
EPOCHS = 40